# Building a comprehensive test suite for a simple function (with examples in python)

## Aim: build a comprehensive test suite for a "greatest common divisor" function, `gcd`

> Definition: the **greatest common divisor** (GCD) of two or more integers, which are not all zero, is the largest positive integer that divides each of the integers. 
> For two integers $x$, $y$, the greatest common divisor of $x$ and $y$ is denoted
$\displaystyle \gcd(x, y)$.
>
> For example, $\displaystyle \gcd(8, 12) = 4$.
>
> ... $\displaystyle \gcd(0, 0)$ is commonly defined as $0$ [*and this is our definition today*].
>
> _Source: Wikipedia, https://en.wikipedia.org/wiki/Greatest_common_divisor, retrieved on September 21 2023_

In [1]:
# import our gcd function from our example module
from gcd import gcd

# Import some other useful utilities
import pytest
import copy

## Essential tooling: `assert` lets us throw an error if a check is false

We can use assert statements in python to write tests.

The assert statement in python raises an `Error` if the result of a calculation is "Falsy":

In [2]:
assert True  # no Error

In [3]:
assert False  # raises an Exception

AssertionError: 

In [4]:
assert 1 == 1  # no Error

In [5]:
assert 1 > 0  # no Error

In [6]:
assert 0 > 1  # False, so raises an exception

AssertionError: 

In [7]:
assert "a"  # a string is truthy...

In [8]:
assert ""  # but an empty string is falsy

AssertionError: 

> Beware: `assert` is meant for debugging, and can be turned off by running `python` with the `-O` flag.
> Use `raise` statements and conditions if your code relies on the check being run.

## We want to check for correct functionality with **good data**

### Type test

Does it produce sensible results, like the correct datatype?

In [9]:
assert type(gcd(8, 12)) is int  

... or the correct sign (+ rather than -)?

In [10]:
assert gcd(8, 12) > 0

### Nominal cases

Check for correct result in all "normal" cases.

In [11]:
assert gcd(7, 21) == 7
assert gcd(20, 10) == 10
assert gcd(54, 24) == 6

### Boundary cases

Check for correctness at the boundaries of the domain, or boundaries within parameters.
Checking the boundary means the value on the boundary, just above, and (if valid) just below.

The `gcd` function operates on integers and has a boundary at zero:

In [12]:
assert gcd(1, 17) == 1  # should be 1
assert gcd(0, 17) == 0  # should be 0

> Python doesn't have a bound on the size of integers, and we'll look at common errors with large values later.

### Special cases

Check behavior at special values (if any exist):

In [13]:
assert gcd(0, 0) == 0

### Symmetries

We also know that $\displaystyle \gcd(x, y) = \displaystyle \gcd(y, x)$ so we should test those too:

In [14]:
# Nominal
assert gcd(21, 7) == 7
assert gcd(10, 20) == 10
assert gcd(24, 54) == 6

# Boundary
assert gcd(17, 1) == 1  # should be 1
assert gcd(17, 0) == 0  # should be 0

## It is vital to test that our function also *fails* correctly for **bad data**



### Incorrect type

If we pass in the wrong kind of data, it should also fail:

In [15]:
with pytest.raises(TypeError):
    gcd(1.2, 2.4)

### Too little data

Analogously, if we pass in too little data:

In [16]:
with pytest.raises(TypeError):
    gcd()

In [17]:
with pytest.raises(TypeError):
    gcd(0)

### Too much data

If we pass in too much data:

In [18]:
gcd(1, 2, 3)  # throws a type error

TypeError: gcd() takes 2 positional arguments but 3 were given

In [34]:
with pytest.raises(TypeError):  # ... which we catch like this
    gcd(1, 2, 3)

## Practically, you might be able to run tests which are disproportionately likely to show errors

Some input values cause more errors than others. 

You might be able to guess which errors will crop up, and test more effectively by finding errors faster.

### Zeros
Zeros often cause problems.

In [22]:
assert gcd(0, 100) == 0

### Values at the limit of a type's definition may cause issues

The "natural" maximum size of an integer might be $2^{63} - 1$ on a 64-bit system, so we'll check that.

> As of python 3, the only size limit for an integer is the size of memory [[1]](https://docs.python.org/3/library/sys.html#sys.maxsize). 

In [35]:
a = 2**63-1  # prime factors: 7, 73, 127, 337, 92737, 649657, https://www.wikidata.org/wiki/Q10571632
b = 649657 * 7 * 6  # the gcd is 649657 * 7 by construction
assert gcd(a, b) == 649657 * 7

### Mutable datatypes can cause very strange errors

In python, it's easy to introduce a fault which causes function to change its output each time you run it, even with the same inputs – check that a function returns the same output for the same input:

In [24]:
# Example of a function which displays this behavior
def append(value, the_list=[]):
    """
    Appends a value to a list, and if the list isn't given, return the value on a new list
    """
    the_list.append(value)
    return the_list

# Works fine if we give it a list to extend:
append(1, [])  # should return [1]

[1]

In [25]:
assert append(1, []) == [1]

If we don't give it a list to extend, it breaks:

In [26]:
assert append(1) == [1]

In [27]:
assert append(2) == [2]   # should return [2]!!!

AssertionError: 

What's going on? Let's try to debug this function:

In [28]:
append(2)

[1, 2, 2]

The default value of `the_list` is getting extended each time we run the function.

You might think that you can check this by running a test like this:

In [29]:
assert append(3) == append(3)  # passes unexpectedly!

... but the error is so insidious that this test fails! Both functions are appending to the same list! 
You actually need to store a copy of the value from the first run and compare it later:

In [30]:
first_result = copy.deepcopy(append(4))
second_result = copy.deepcopy(append(4))

assert first_result == second_result, "%s != %s" % (first_result, second_result)

AssertionError: [1, 2, 2, 3, 3, 4] != [1, 2, 2, 3, 3, 4, 4]

To fix this, we replace the mutable list in the function with a `None`:

In [36]:
def append_fixed(value, the_list=None):
    """
    Appends a value to a list, and if the list isn't given, return the value on a new list
    """
    if the_list is None:
        the_list = []
    the_list.append(value)
    return the_list

first_result = copy.deepcopy(append_fixed(4))
second_result = copy.deepcopy(append_fixed(4))

assert first_result == second_result, "%s != %s" % (first_result, second_result)

In the context of our `gcd` function, the test would be:

In [31]:
first_gcd = copy.deepcopy(gcd(32, 8))
second_gcd = copy.deepcopy(gcd(32, 8))

assert first_gcd == second_gcd, "%s != %s" % (first_gcd, second_gcd)

### A test for every bug – regression tests

A rich source of errors is *faults which were already fixed*.

These faults can re-emerge, and are then called "regressions". 

Solution: Every time you find a bug: 

- Make a test case which fails because of the bug.
- Fix the bug (so the test case passes)
- Leave the test case in your testing library.

## Organizing tests

### Smoke test

Check for basic plausibility. Does it run without failing?

In [20]:
gcd(8, 12)  # runs without failing

4

Does it produce the same result if a and b are swapped? (Only true for commutative operations)

In [21]:
assert gcd(12, 8)

## The "black-box" approach varies the inputs to a function and checks its outputs

<Image of a black box here>

The first approach we'll use is the "black-box" approach, which treats the function as a black box which we can't see inside. 

We'll look at inputs and check that they produce the correct outputs.